Criando um ambiente confiável para avaliar nosso modelo

No ML o que importa é a performance em dados em que seu modelo nunca viu

1. Treino (o modelo aprende os pesos, onde dividir os dados e ajustar os parâmetros)
2. Validação (testa o modelo treinado em dados que ele nunca viu, ou seja, uma simulação da realidade)
3. Teste (só é avaliado quando se tem um modelo  final)

In [1]:
# importando as bibliotecas
import pandas as pd
import numpy as np

In [2]:
# Função para transformar a coluna sexo em 0 e 1
def transformar_sexo(valor):
    if valor == 'female':
        return 1
    else:
        return 0

In [4]:
train = pd.read_csv("train.csv") # lendo o arquivo train.csv
test = pd.read_csv("test.csv") # lendo o arquivo test.csv

train['Sex_binario'] = train['Sex'].map(transformar_sexo) # aplicando a função transformar_sexo
test['Sex_binario'] = test['Sex'].map(transformar_sexo) # aplicando a função transformar_sexo

variaveis = ['Sex_binario', 'Age'] # variáveis que serão usadas para treinar o modelo

X = train[variaveis].fillna(-1) # preenchendo os valores nulos com -1
y = train['Survived'] # variável que será prevista

In [6]:
train.head() # exibindo as primeiras linhas do dataframe

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_binario
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [7]:
from sklearn.ensemble import RandomForestClassifier # importando o classificador Random Forest
from sklearn.model_selection import train_test_split # importando a função para dividir o dataset em treino e teste

In [9]:
X_falso = np.arange(10) # criando um array de 10 elementos
X_falso # exibindo o array

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [11]:
np.random.seed(0)   # para garantir que os resultados sejam os mesmos
train_test_split(X_falso, test_size=0.5) # dividindo o dataset em treino e teste

[array([6, 7, 3, 0, 5]), array([2, 8, 4, 9, 1])]

In [12]:
np.random.seed(1) # definindo a semente para garantir que os resultados sejam os mesmos
X_treino, X_valid, y_treino, y_valid = train_test_split(X, y, test_size=0.5) # dividindo o dataset em treino e teste

In [13]:
X_treino.head() # exibindo as primeiras linhas do dataframe

,Sex_binario,Age
394,1,24.00
851,0,74.00
373,0,22.00
523,1,44.00
78,0,0.83


In [14]:
X_treino.shape, X_valid.shape, y_treino.shape, y_valid.shape # exibindo o formato dos datasets

((445, 2), (446, 2), (445,), (446,))

In [15]:
modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0) # criando o modelo Random Forest
modelo.fit(X_treino, y_treino)   # treinando o modelo

RandomForestClassifier(n_jobs=-1, random_state=0)

In [16]:
p = modelo.predict(X_valid)

In [17]:
np.mean(y_valid == p)

0.7466367713004485

In [30]:
p = (X_valid['Sex_binario'] == 1).astype(np.int64)  # criando um modelo que prevê que todas as mulheres sobreviveram
# y_valid == p  comparando as previsões com os valores reais
np.mean(y_valid == p) # calculando a acurácia do modelo

0.7921348314606742

## Validação cruzada

In [19]:
X_falso

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [20]:
from sklearn.model_selection import KFold # importando a função KFold

In [32]:
# Existem validações cruzadas mais sofisticadas, como StratifiedKFold e GroupKFold. Usa-se para séries temporais, por exemplo, TimeSeriesSplit.


# criando um objeto KFold
kf = KFold(3, shuffle=True, random_state=0) # dividindo o dataset em 3 partes, shuffle=True para embaralhar as linhas, random_state=0 para garantir que os resultados sejam os mesmos
for linhas_treino, linhas_valid in kf.split(X_falso): # iterando sobre as divisões
    print("Treino:", linhas_treino) # exibindo as linhas de treino
    print("Valid:", linhas_valid)   # exibindo as linhas de validação
    print() # exibindo as linhas de validação

Treino: [0 1 3 5 6 7]
Valid: [2 4 8 9]

Treino: [0 2 3 4 5 8 9]
Valid: [1 6 7]

Treino: [1 2 4 6 7 8 9]
Valid: [0 3 5]



In [33]:
# Existe uma função que faz isso automaticamente, o cross_val_score. Ela já divide o dataset e calcula a acurácia do modelo. Mas, como estamos aprendendo, vamos fazer manualmente.

resultados = [] # criando uma lista vazia para armazenar os resultados
for rep in range(10): # iterando 10 vezes
    print("Rep:", rep)  # exibindo o número da iteração
    kf = KFold(5, shuffle=True, random_state=rep) # criando um objeto KFold
    
    for linhas_treino, linhas_valid in kf.split(X): # iterando sobre as divisões
        print("Treino:", linhas_treino.shape[0]) # exibindo o número de linhas de treino
        print("Valid:", linhas_valid.shape[0])  # exibindo o número de linhas de validação

        X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid] # dividindo o dataset X
        y_treino, y_valid = y.iloc[linhas_treino], y.iloc[linhas_valid] # dividindo o dataset y

        modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0) # criando o modelo Random Forest
        modelo.fit(X_treino, y_treino)  # treinando o modelo

        p = modelo.predict(X_valid) # fazendo as previsões

        acc = np.mean(y_valid == p) # calculando a acurácia
        resultados.append(acc) # armazenando a acurácia na lista de resultados
        print("Acc:", acc) # exibindo a acurácia
        print() # exibindo uma linha em branco
        #print(X_treino.head())
        #print()

Rep: 0
Treino: 712
Valid: 179
Acc: 0.7988826815642458

Treino: 713
Valid: 178
Acc: 0.7359550561797753

Treino: 713
Valid: 178
Acc: 0.7808988764044944

Treino: 713
Valid: 178
Acc: 0.797752808988764

Treino: 713
Valid: 178
Acc: 0.7808988764044944

Rep: 1
Treino: 712
Valid: 179
Acc: 0.7374301675977654

Treino: 713
Valid: 178
Acc: 0.7247191011235955

Treino: 713
Valid: 178
Acc: 0.7808988764044944

Treino: 713
Valid: 178
Acc: 0.7921348314606742

Treino: 713
Valid: 178
Acc: 0.7921348314606742

Rep: 2
Treino: 712
Valid: 179
Acc: 0.7653631284916201

Treino: 713
Valid: 178
Acc: 0.7865168539325843

Treino: 713
Valid: 178
Acc: 0.7865168539325843

Treino: 713
Valid: 178
Acc: 0.7808988764044944

Treino: 713
Valid: 178
Acc: 0.7640449438202247

Rep: 3
Treino: 712
Valid: 179
Acc: 0.7653631284916201

Treino: 713
Valid: 178
Acc: 0.7471910112359551

Treino: 713
Valid: 178
Acc: 0.7808988764044944

Treino: 713
Valid: 178
Acc: 0.7415730337078652

Treino: 713
Valid: 178
Acc: 0.8202247191011236

Rep: 4
Treino

In [34]:
len(resultados) # exibindo o tamanho da lista de resultados

50

In [35]:
np.mean(resultados) # calculando a média dos resultados

0.7692593057560732

## Criar submission

In [37]:
sub = pd.Series(p, index=test['PassengerId'][:178], name='Survived') # criando um dataframe com as previsões
sub.shape   # exibindo o formato do dataframe

(178,)

In [44]:
sub.to_csv("primeiro_modelo.csv", header=True)  # salvando o dataframe em um arquivo csv

In [45]:
!head -n10 primeiro_modelo.csv # exibindo as 10 primeiras linhas do arquivo csv

'head' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [46]:
# Lendo as primeiras 10 linhas do arquivo CSV
df = pd.read_csv('primeiro_modelo.csv', nrows=10)
print(df)

   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         1
4          896         1
5          897         0
6          898         0
7          899         0
8          900         0
9          901         1
